# Chapter 3: Assigning Roles & Role Prompting

This lesson explores how to use system, user, and assistant roles to guide Ollama's behavior and responses.

- What are roles in prompt engineering?
- How to use system prompts to set context
- How to simulate multi-turn conversations
- Exercises to practice role prompting

Run the setup cell below to initialize your Ollama environment.

In [ ]:
# Run environment setup (customized for your env)
def pretty_print_response():
    return
def get_completion():
    return
%run 00_Tutorial_How-To.ipynb

## What Are Roles?

Roles help you control the context and style of Ollama's responses.
- **system**: Sets the behavior or persona of the model
- **user**: Represents the user's input
- **assistant**: Represents the model's previous responses

You can use these roles to simulate conversations, set constraints, or guide the model's tone.

## Example: Setting a System Role
Let's make Ollama answer as a friendly chef.

In [ ]:
# First, call without a system prompt to see the default response
prompt = 'How do I make a perfect omelette?'
pretty_print_response(get_completion(prompt))

# Now, call with a system prompt to see the difference
system_prompt = 'You are a friendly chef who loves to share cooking tips.'
pretty_print_response(get_completion(prompt, system_prompt=system_prompt))

## Example: Multi-Turn Conversation
You can provide a list of messages to simulate a conversation with different roles.

In [ ]:
conversation = [
    {"role": "system", "content": "You are a travel expert specializing in France."},
    {"role": "user", "content": "What are the top sights in Paris?"},
    {"role": "assistant", "content": "Some top sights in Paris include the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral."},
    {"role": "user", "content": "What is the best time to visit?"}
]
pretty_print_response(get_completion(conversation))

## Exercise 1: Create a Role
Write a system prompt that makes Ollama answer as a famous scientist.
Try asking a science question and see how the response changes.

In [7]:
# System prompt - change this
system_prompt = 'you are albert einstein assistant, and you like to give very short answers in a sassy tone.'
prompt = 'What is the speed of light?'
pretty_print_response(get_completion(prompt, system_prompt=system_prompt))

Model: qwen2.5:14b-instruct
Created at: 2025-08-17 14:26:59 UTC
Done: True
Done reason: stop
Total duration: 2.01 s
Load duration: 0.19 s
Prompt eval count: 42
Prompt eval duration: 0.46 s
Eval count: 20
Eval duration: 1.36 s
Message:
  Role: assistant
  Content: Approximately 299,792 kilometers per second, but who's counting?


## Exercise 2: Multi-Turn Role Play
Simulate a conversation between a chef and a user asking for a recipe.
Edit the conversation below to experiment with different roles and messages.

In [ ]:
conversation = [
    {"role": "system", "content": "you are a korean chef who makes vegan omelette."},
    {"role": "user", "content": "Can you share a pasta recipe?"}
]
pretty_print_response(get_completion(conversation))